In [2]:
!pip install pandas scikit-learn torch datasets transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 24.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import os
# Hugging Face Transformers and Datasets
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    AutoConfig
)

# -------------------------------
# Configuration
# -------------------------------
Project_Base_Path = '/content/'  # Ensure this directory exists or adjust as needed
MODEL_NAME = 'cardiffnlp/twitter-roberta-base-sentiment-latest'

# Define Save Directory and Logs Directory
SAVE_DIR = os.path.join(Project_Base_Path, "saved_models")  # Correctly join paths
LOGS_DIR = os.path.join(Project_Base_Path, "logs")        # Define logs directory

# Create directories if they do not exist
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)
print(f"Directories '{SAVE_DIR}' and '{LOGS_DIR}' are ready.")

# -------------------------------
# Load dataset
# -------------------------------
# Load the dataset once
df = pd.read_parquet("hf://datasets/TimKoornstra/synthetic-financial-tweets-sentiment/data/train-00000-of-00001.parquet")
print("Dataset loaded successfully.")

# -------------------------------
# Assign 'label' Column
# -------------------------------
# Since 'sentiment' is already numerical, assign it directly to 'label'
df['label'] = df['sentiment']

# -------------------------------
# Inspect the DataFrame
# -------------------------------
# Display the first few rows to understand the data structure
print("\nFirst few rows of the DataFrame:")
print(df.head())

# Check for missing values in the 'sentiment' column
num_missing_sentiment = df["sentiment"].isna().sum()
print(f"\nNumber of missing 'sentiment' values: {num_missing_sentiment}")

# Display unique sentiment labels
unique_sentiments = df["sentiment"].unique()
print(f"Unique sentiment labels before mapping: {unique_sentiments}")

# -------------------------------
# Validate the 'label' Column
# -------------------------------
# Check for missing values in the 'label' column
num_missing_label = df["label"].isna().sum()
print(f"Number of missing 'label' values: {num_missing_label}")

if num_missing_label > 0:
    print("Removing rows with missing 'label' values...")
    df = df.dropna(subset=["label"]).reset_index(drop=True)
    print(f"Data shape after removing NaNs: {df.shape}")
else:
    print("No missing 'label' values found.")

# -------------------------------
# Define Label Mapping for Model Configuration
# -------------------------------
# Define mappings for clarity in model configuration
id2label = {
    0: "Negative",
    1: "Neutral",
    2: "Positive"
}

label2id = {
    "Negative": 0,
    "Neutral": 1,
    "Positive": 2
}

# -------------------------------
# Split into Train, Validation, and Test Sets
# -------------------------------
# First split: Train + Temp (Validation + Test)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df["tweet"],
    df["label"],
    test_size=0.3,  # 70% train, 30% temp
    stratify=df["label"],  # Ensures balanced classes in splits
    random_state=42
)

# Second split: Validation + Test from Temp
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts,
    temp_labels,
    test_size=0.5,  # 15% validation, 15% test
    stratify=temp_labels,
    random_state=42
)

print(f"\nTraining set size: {len(train_texts)}")
print(f"Validation set size: {len(val_texts)}")
print(f"Test set size: {len(test_texts)}")

# -------------------------------
# Convert into Hugging Face Datasets
# -------------------------------
train_dataset = Dataset.from_dict({
    "text": train_texts.tolist(),
    "label": train_labels.tolist()
})
val_dataset = Dataset.from_dict({
    "text": val_texts.tolist(),
    "label": val_labels.tolist()
})
test_dataset = Dataset.from_dict({
    "text": test_texts.tolist(),
    "label": test_labels.tolist()
})

print("\nHugging Face Datasets created successfully.")

# -------------------------------
# Tokenizer
# -------------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("Tokenizer loaded successfully.")

def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

# Apply tokenization
train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)

# Set dataset format for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

print("Datasets tokenized and formatted for PyTorch.")





Directories '/content/saved_models' and '/content/logs' are ready.
Dataset loaded successfully.

First few rows of the DataFrame:
                                               tweet  sentiment  label
0  💰 Cashing out stocks today has left me with a ...          2      2
1  💸 Losing half my investments in the last month...          2      2
2  📉 Crypto bubble finally bursts! The crypto tra...          2      2
3  🔥 The blazing inferno engulfs the stock market...          2      2
4  🥶 My stocks are stuck in an icy grip today. Fi...          2      2

Number of missing 'sentiment' values: 0
Unique sentiment labels before mapping: [2 0 1]
Number of missing 'label' values: 0
No missing 'label' values found.

Training set size: 1000139
Validation set size: 214316
Test set size: 214316

Hugging Face Datasets created successfully.
Tokenizer loaded successfully.


Map:   0%|          | 0/1000139 [00:00<?, ? examples/s]

Map:   0%|          | 0/214316 [00:00<?, ? examples/s]

Map:   0%|          | 0/214316 [00:00<?, ? examples/s]

Datasets tokenized and formatted for PyTorch.


In [6]:
# -------------------------------
# Load Pretrained Model with Correct Label Mapping
# -------------------------------
# Define the configuration with label mappings
config = AutoConfig.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    config=config
)
print("Pretrained model loaded with correct label mappings.")

# -------------------------------
# (Optional) Freeze Base RoBERTa Layers
# So Only Classification Head is Trained
# -------------------------------
# Freeze all RoBERTa base model parameters
for param in model.roberta.parameters():
    param.requires_grad = False

print("RoBERTa base layers frozen.")

# -------------------------------
# Training Arguments
# -------------------------------
LEARNING_RATE = 5e-5
EPOCHS = 1
BATCH_SIZE = 256
SAVE_DIR = f"{Project_Base_Path}saved_models"  # Removed space between '/' and 'saved_models'

training_args = TrainingArguments(
    output_dir=SAVE_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",        # Save model each epoch
    logging_dir="logs",           # Where logs are stored
    logging_steps=50,             # Log every 50 steps
    load_best_model_at_end=True,  # In case you want the best checkpoint
    metric_for_best_model="accuracy",  # Define which metric to use for best model
    greater_is_better=True
)

print("Training arguments configured.")

# -------------------------------
# Define Metrics (Optional but Recommended)
# -------------------------------
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# -------------------------------
# Create Trainer
# -------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,  # Pass the metrics function
)

print("Trainer initialized.")

# -------------------------------
# Fine-tune (Train)
# -------------------------------
trainer.train()

print("Training completed.")

# -------------------------------
# Save the Model
# -------------------------------
trainer.save_model(SAVE_DIR)
print(f"Model saved to {SAVE_DIR}.")

# -------------------------------
# Evaluate on Test Set (Optional)
# -------------------------------
test_results = trainer.evaluate(test_dataset)
print("\nTest Set Evaluation Results:")
print(test_results)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Pretrained model loaded with correct label mappings.
RoBERTa base layers frozen.
Training arguments configured.
Trainer initialized.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.289600,0.256618,0.899536,0.899509,0.899536,0.899478


Training completed.
Model saved to /content/saved_models.



Test Set Evaluation Results:
{'eval_loss': 0.25621935725212097, 'eval_accuracy': 0.8994428787398048, 'eval_precision': 0.8993478598217183, 'eval_recall': 0.8994428787398048, 'eval_f1': 0.8993547127749089, 'eval_runtime': 1334.3086, 'eval_samples_per_second': 160.62, 'eval_steps_per_second': 0.628, 'epoch': 1.0}
